In [35]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [36]:
base_dir = 'storage/output/220314_baseline/'
chucksize = 1000
k = 10

In [37]:
%%time 
transactions = pd.read_csv('storage/transactions_train.csv')
submission = pd.read_csv('storage/sample_submission.csv')

transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])

CPU times: user 24.4 s, sys: 1.79 s, total: 26.2 s
Wall time: 26.2 s


In [39]:
from utils import train_val_test_split
trn_transactions,val_transactions,test_transactions = train_val_test_split(transactions)

/home/kinho.lo/kaggle/kaggle-h-and-m-personalized-fashion-recommendation/utils.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['t_dat'] = pd.to_datetime(df['t_dat'])


In [60]:
print(trn_transactions.t_dat.max(),val_transactions.t_dat.max(),test_transactions.t_dat.max())
transactions_3w = trn_transactions[trn_transactions['t_dat'] >= pd.to_datetime('2020-08-17')].copy()
transactions_2w = trn_transactions[trn_transactions['t_dat'] >= pd.to_datetime('2020-08-24')].copy()
transactions_1w = trn_transactions[trn_transactions['t_dat'] >= pd.to_datetime('2020-08-31')].copy()

2020-09-08 00:00:00 2020-09-15 00:00:00 2020-09-22 00:00:00


In [50]:
purchase_dict_3w = {}

for i,x in enumerate(zip(transactions_3w['customer_id'], transactions_3w['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict_3w:
        purchase_dict_3w[cust_id] = {}
    
    if art_id not in purchase_dict_3w[cust_id]:
        purchase_dict_3w[cust_id][art_id] = 0
    
    purchase_dict_3w[cust_id][art_id] += 1
    
print(len(purchase_dict_3w))

dummy_list_3w = list((transactions_3w['article_id'].value_counts()).index)[:12]

203316


In [51]:
purchase_dict_2w = {}

for i,x in enumerate(zip(transactions_2w['customer_id'], transactions_2w['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict_2w:
        purchase_dict_2w[cust_id] = {}
    
    if art_id not in purchase_dict_2w[cust_id]:
        purchase_dict_2w[cust_id][art_id] = 0
    
    purchase_dict_2w[cust_id][art_id] += 1
    
print(len(purchase_dict_2w))

dummy_list_2w = list((transactions_2w['article_id'].value_counts()).index)[:12]

157433


In [52]:
purchase_dict_1w = {}

for i,x in enumerate(zip(transactions_1w['customer_id'], transactions_1w['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict_1w:
        purchase_dict_1w[cust_id] = {}
    
    if art_id not in purchase_dict_1w[cust_id]:
        purchase_dict_1w[cust_id][art_id] = 0
    
    purchase_dict_1w[cust_id][art_id] += 1
    
print(len(purchase_dict_1w))

dummy_list_1w = list((transactions_1w['article_id'].value_counts()).index)[:12]

94257


In [53]:
pred_df = submission[['customer_id']]
prediction_list = []

dummy_pred = list((transactions_1w['article_id'].value_counts()).index)[:12]

for i, cust_id in enumerate(submission['customer_id'].values.reshape((-1,))):
    s = []
    if cust_id in purchase_dict_1w:
        l = sorted((purchase_dict_1w[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
            s += l[:12]
        else:
            s += l
    if cust_id in purchase_dict_2w:
        l = sorted((purchase_dict_2w[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
            s += l[:12]
        else:
            s += l
    if cust_id in purchase_dict_1w:
        l = sorted((purchase_dict_1w[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
            s += l[:12]
        else:
            s += l
    s += dummy_pred
    prediction_list.append(s)

pred_df['prediction'] = prediction_list
print(pred_df.shape)
pred_df.head()

(1371980, 2)


,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"[568601043, 568601043, 568601043, 915526001, 7..."
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,"[915526001, 751471001, 751471043, 706016001, 9..."
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[915526001, 751471001, 751471043, 706016001, 9..."
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,"[915526001, 751471001, 751471043, 706016001, 9..."
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,"[915526001, 751471001, 751471043, 706016001, 9..."


In [57]:
gt_df = val_transactions.groupby('customer_id')['article_id'].agg(lambda x: x.tolist()).reset_index()
gt_df.columns = ['customer_id','ground_truth']

In [58]:
def evaluate_score(pred_df,gt_df,k=12):
    from metric import mapk
    eval_df = gt_df.merge(pred_df,on='customer_id')
    print('map@'+str(k), mapk(eval_df['ground_truth'].tolist(),eval_df['prediction'].tolist()))

In [59]:
evaluate_score(pred_df,gt_df)

map@12 0.020237018639026803


In [30]:
cand_df = pred_df.explode('prediction')

In [63]:
cand_df = cand_df.rename(columns={'prediction':'article_id'})

In [64]:
cand_df.to_csv(os.path.join(base_dir,'candidate_df.csv'))